In [1]:
from Detik import Detik #class news Scrapper
from datetime import timedelta, date, datetime
from IPython.display import clear_output

In [2]:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [3]:
obj = Detik()
end_date = date(2018, 8, 12)
start_date = date(2018, 1, 1)
init_page = 1
list_cat = [['news', 'news'], ['oto', 'otomotif'], ['travel', 'travel'], ['finance', 'finance']
           , ['hot', 'hot'], ['inet', 'inet'], ['sport', 'sport'], ['food', 'food']
           , ['health', 'health']]
init_cat = list_cat[8:]

In [ ]:
for single_date in daterange(start_date, end_date):
    for cat in init_cat:
        clear_output()
        print(single_date.strftime("%Y-%m-%d"))
        print(cat[0])
        obj.getAllBerita([], init_page, cat[0], cat[1], single_date.strftime("%d/%m/%Y"))
        init_page = 1
    init_cat = list_cat

2018-01-02
news
page  1
https://news.detik.com/indeks/all/1?date=02/01/2018
https://news.detik.com/berita-jawa-barat/d-3845875/cerita-saksi-saat-pelaku-ngamuk-dan-pukul-komandan-brigade-persis
memasukkan berita id  3845875
Insert berita  
masuk
https://news.detik.com/berita/d-3845871/saksi-tak-tahu-soal-saham-murakabi-jaksa-kan-anda-direktur
memasukkan berita id  3845871
Insert berita  
masuk
https://news.detik.com/berita-jawa-barat/d-3845864/kawanan-perampok-libatkan-wanita-hamil-1-pelaku-ditembak-mati
memasukkan berita id  3845864
Insert berita  
masuk
https://news.detik.com/berita/d-3845862/gandeng-mui-sandiaga-cari-2-destinasi-untuk-dijadikan-wisata-halal
memasukkan berita id  3845862
Insert berita  
masuk
https://news.detik.com/berita-jawa-timur/d-3845848/praktik-pengoplosan-elpiji-di-gresik-polisi-ringkus-2-pelaku
memasukkan berita id  3845848
Insert berita  
masuk
https://news.detik.com/berita/d-3845849/jembatan-mengerikan-di-jagakarsa-akan-dibangun-pekan-depan
memasukkan berita

masuk
foto news
https://news.detik.com/berita/d-3845611/dijaga-satpol-pp-trotoar-sudirman-tak-lagi-dipenuhi-pkl
memasukkan berita id  3845611
Insert berita  
masuk
https://news.detik.com/berita/d-3845610/baleg-prioritaskan-ruu-md3-sebelum-selesaikan-ruu-penyiaran
memasukkan berita id  3845610
Insert berita  
masuk
page  4
https://news.detik.com/indeks/all/4?date=02/01/2018
https://news.detik.com/berita/d-3845597/pansus-angket-minta-jokowi-bentuk-pengawas-ini-kata-kpk
memasukkan berita id  3845597
Insert berita  
masuk
foto news
sidang eks dirjen hubla
https://news.detik.com/berita/d-3845576/jaksa-cecar-saksi-soal-21-rekening-penyuap-eks-dirjen-hubla
memasukkan berita id  3845576
Insert berita  
masuk
https://news.detik.com/berita/d-3845577/sandi-bantu-sopir-tn-abang-buat-ktp-dki-kemendagri-ada-prosedurnya
memasukkan berita id  3845577
Insert berita  
masuk
https://news.detik.com/berita-jawa-tengah/d-3845567/mahasiswa-amikom-yogya-meninggal-usai-mengikuti-diksar-kampus
memasukkan berita

memasukkan berita id  3845339
Insert berita  
masuk
peristiwa
https://news.detik.com/berita-jawa-timur/d-3845355/ini-tanggapan-bupati-ipong-terkait-demo-pkl-dan-bentor-di-ponorogo
memasukkan berita id  3845355
Insert berita  
masuk
https://news.detik.com/berita/d-3845335/fredrich-yunadi-segera-jalani-sidang-kasus-hilangnya-novanto
memasukkan berita id  3845335
Insert berita  
masuk
https://news.detik.com/bbc-world/d-3845030/fbi-dan-gedung-putih-bentrok-soal-memo-rahasia
memasukkan berita id  3845030
Insert berita  
masuk
https://news.detik.com/berita/d-3845322/menyikapi-kaum-waria-berkaca-kepada-gubernur-ali-sadikin--
memasukkan berita id  3845322
Insert berita  
masuk
pilgub jabar 2018
https://news.detik.com/berita-jawa-barat/d-3845321/kpu-jabar-siapkan-rp-17-miliar-untuk-alat-peraga-kampanye
memasukkan berita id  3845321
Insert berita  
masuk
https://news.detik.com/berita/d-3845332/soal-pidato-kapolri-polri-akan-terus-silaturahmi-ke-ormas-ormas
memasukkan berita id  3845332
Insert be

memasukkan berita id  3845101
Insert berita  
masuk
https://news.detik.com/berita/d-3845105/jk-akan-terima-penghargaan-tokoh-inspiratif-penanganan-bencana
memasukkan berita id  3845105
Insert berita  
masuk
https://news.detik.com/berita-jawa-timur/d-3845073/70-prajurit-tni-di-surabaya-melakukan-pelanggaran-di-tahun-2017
memasukkan berita id  3845073
Insert berita  
masuk
https://news.detik.com/berita-jawa-barat/d-3845067/sumur-dan-makam-tua-di-cirebon-diduga-ada-sejak-abad-ke-14
memasukkan berita id  3845067
Insert berita  
masuk
https://news.detik.com/berita-jawa-barat/d-3845063/4000-jemaah-sbl-tanyakan-nasib-keberangkatan-umrah-ke-polisi
memasukkan berita id  3845063
Insert berita  
masuk
https://news.detik.com/berita/d-3845066/tambah-1-kursi-pimpinan-dpr-ruu-md3-diparipurnakan-8-februari
memasukkan berita id  3845066
Insert berita  
masuk
https://news.detik.com/berita/d-3845070/diduga-stres-ditinggal-pacar-nurhayati-tenggak-detergen
memasukkan berita id  3845070
Insert berita  
masu

masuk
https://news.detik.com/berita/d-3844899/jual-narkoba-an-beli-mini-cooper-dan-tanah-20-hektare
memasukkan berita id  3844899
Insert berita  
masuk
https://news.detik.com/berita-jawa-tengah/d-3844901/kapal-pengangkut-bbm-ke-karimunjawa-masih-bersandar-di-semarang
memasukkan berita id  3844901
Insert berita  
masuk
foto news
https://news.detik.com/berita/d-3844889/alexis-bantah-ada-prostitusi-di-bar-4play
memasukkan berita id  3844889
Insert berita  
masuk
sidang setya novanto
https://news.detik.com/berita/d-3844902/sebut-e-ktp-bermasalah-sejak-awal-lkpp-sudah-peringatkan-gamawan
memasukkan berita id  3844902
Insert berita  
masuk
https://news.detik.com/berita/d-3844890/zulkifli-hasan-siap-maju-sebagai-capres-atau-cawapres-di-2019
memasukkan berita id  3844890
Insert berita  
masuk
https://news.detik.com/berita/d-3844868/prasetyo-dengar-masih-ada-jaksa-pengawal-apbd-yang-korup
memasukkan berita id  3844868
Insert berita  
masuk
https://news.detik.com/berita-jawa-tengah/d-3844865/dig

memasukkan berita id  3844690
Insert berita  
masuk
https://news.detik.com/berita-jawa-barat/d-3844688/warga-cicalengka-temukan-mayat-laki-laki-di-semak-semak
memasukkan berita id  3844688
Insert berita  
masuk
berita foto
page  15
https://news.detik.com/indeks/all/15?date=02/01/2018
https://news.detik.com/berita/d-3844689/anies-akan-ubah-kepgub-soal-pengubahan-nama-jalan-di-jakarta
memasukkan berita id  3844689
Insert berita  
masuk
https://news.detik.com/berita/d-3844692/jokowi-terima-perwira-tni-polri-yang-akan-naik-pangkat
memasukkan berita id  3844692
Insert berita  
masuk
kolom
https://news.detik.com/kolom/d-3844665/penganiayaan-kiai-basri-dan-empat-masalah-utama-umat-hari-ini
memasukkan berita id  3844665
Insert berita  
masuk
sidang setya novanto
https://news.detik.com/berita/d-3844685/di-sidang-novanto-hakim-semua-bantah-tapi-jelas-ada-yang-bohong
memasukkan berita id  3844685
Insert berita  
masuk
foto news
https://news.detik.com/berita-jawa-tengah/d-3844655/2-dari-3-tangkapa